## Import

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [15, 15]
mpl.rcParams['figure.dpi'] = 72

import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
from utils import *

In [ ]:
img_name="timecity_01886_none"
img = cv2.imread(f'./svm_for_shadow_generation/images/{img_name}.jpg')
org_mask = cv2.imread(f'./svm_for_shadow_generation/masks/{img_name}.jpg')
data = json.load(open(f'./svm_for_shadow_generation/leg_pos/{img_name}.json', "r"))
fixed_points = np.zeros((1,4,2)) # leg positions
for i in range(4):
    fixed_points[0][i] = data['shapes'][i]['points'][0]

In [ ]:
is_match_legs = False
if is_match_legs is False:
    is_vertical_flip = np.random.choice(a=[False, True])
else:
    is_vertical_flip = False
is_vertical_flip = True
if is_match_legs is True and is_vertical_flip is True:
    assert False, "Can not flip shadow while matching legs with TPS"

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
background=cv2.subtract(img,org_mask)
cut_obj=cv2.subtract(img,background)

In [ ]:
def mask_to_shadow2(mask, fixed_points, rot_rd_low, rot_rd_high, is_vertical_flip = False, match_legs=False):
    center_x = (min(fixed_points[0,:,0]) + max(fixed_points[0,:,0]))/2
    center_y = (min(fixed_points[0,:,1]) + max(fixed_points[0,:,1]))/2
    center_coordinates = np.array([center_x, center_y]).astype('int')

    tl, tr, br, bl = order_points(fixed_points[0])
    mid_l = (tl+bl)/2
    mid_r = (tr+br)/2
    angle = np.arctan((mid_r-mid_l)[1]/(mid_r-mid_l)[0])*180/np.pi
    
    # Rotate image
    h, w = mask.shape[0], mask.shape[1]
    rot_rd = np.random.randint(low = rot_rd_low, high = rot_rd_high)
    rotate_matrix_x = get_rotate_matrix(h, w, alpha=90+rot_rd, beta=90, gamma=90, dx=0, dy=0, dz=200, f=200)
    rotated_x = cv2.warpPerspective(mask, rotate_matrix_x, (h, w), cv2.INTER_LANCZOS4)
    
    mid_pts = np.expand_dims(np.vstack((mid_l, mid_r)), axis=0)
    rotated_mid_pts = cv2.perspectiveTransform(mid_pts, rotate_matrix_x)
    rotated_mid_l = rotated_mid_pts[0,0]
    rotated_mid_r = rotated_mid_pts[0,1]
    rotated_angle = np.arctan((rotated_mid_r-rotated_mid_l)[1]/(rotated_mid_r-rotated_mid_l)[0])*180/np.pi
    rotate_matrix_z = get_rotate_matrix(h, w, alpha=90, beta=90, gamma=90+angle-rotated_angle, dx=0, dy=0, dz=200, f=200)
    rotated = cv2.warpPerspective(rotated_x, rotate_matrix_x, (h, w), cv2.INTER_LANCZOS4)
    rotate_matrix = np.matmul(rotate_matrix_x, rotate_matrix_z)
    
    # Translate image so the shadow touch the legs
    # Translate after rotate x-axis or it won't be correct
    if not match_legs:
        # Affine transform
        rd = np.random.randint(low = -h//8, high = h//8, size=2)
        srcTri = np.array( [[0,0], [w-1,0], [w//2, h//2]]).astype(np.float32)
        dstTri = np.array( [[0,0], [w-1,0], [w//2, h//2]]).astype(np.float32)
        # TODO: bug here
        # dstTri = np.array( [[0,0], [w-1,0], [w//2+rd[0], h//2+rd[1]]]).astype(np.float32)
        warp_mat = cv2.getAffineTransform(srcTri, dstTri)        
        warp_matrix = np.zeros_like(rotate_matrix)
        warp_matrix[2,2] = 1
        warp_matrix[:2] = warp_mat
        warp_dst = cv2.warpAffine(rotated, warp_mat, (w, h))
        
        # Matching shadow
        l, r, t, b = get_mask_bbox(mask)
        new_l, new_r, new_t, new_b = get_mask_bbox(warp_dst)
        # matching the x-axis center of 2 bboxes
        dx=int(l+(r-l)/2-new_l-(new_r-new_l)/2)

        # Flip shadow
        if not is_vertical_flip:
            # matching the bottom of 2 bboxes
            dy=int(b-new_b)
        else:
            rotate_matrix = get_rotate_matrix(h, w, alpha=90+rot_rd, beta=90, gamma=90-angle, dx=0, dy=0, dz=200, f=200)
            rotated = cv2.warpPerspective(mask, rotate_matrix, (h, w), cv2.INTER_LANCZOS4)
            
            # warp_dst = cv2.flip(warp_dst, 0)
            x_flip_matrix = np.eye(3)
            x_flip_matrix[1,1]=-1
            x_flip_matrix[1,2]=h-1
            flipped = cv2.warpPerspective(rotated, x_flip_matrix, (h, w), cv2.INTER_LANCZOS4)
            
            return_rotate_matrix = get_rotate_matrix(h, w, alpha=90+rot_rd, beta=90, gamma=90+angle, dx=0, dy=0, dz=200, f=200)
            return_rotate = cv2.warpPerspective(flipped, return_rotate_matrix, (h, w), cv2.INTER_LANCZOS4)
            
            warp_dst = cv2.warpAffine(return_rotate, warp_mat, (w, h))
            new_l, new_r, new_t, new_b = get_mask_bbox(warp_dst)
            # matching the x-axis center of 2 bboxes
            dx=int(l+(r-l)/2-new_l-(new_r-new_l)/2)
            
            all_matrix = np.matmul(rotate_matrix, x_flip_matrix)
            all_matrix = np.matmul(all_matrix, return_rotate_matrix)
            all_matrix = np.matmul(all_matrix, warp_matrix)
            final_points = cv2.perspectiveTransform(np.expand_dims(center_coordinates, axis=(0,1)).astype('float64'),
                                                           all_matrix)
            dy=int(center_y-final_points[0,0,1])
        trans_matrix = get_rotate_matrix(h, w, alpha=90, beta=90, gamma=90,
                                         dx=dx, dy=dy, dz=200, f=200) # does not translate correctly, pixel unit wrong?
        warped = cv2.warpPerspective(warp_dst, trans_matrix, (h, w), cv2.INTER_LANCZOS4)
        return warped, np.matmul(rotate_matrix, warp_matrix), trans_matrix
    else:
        return rotated, rotate_matrix, None

In [ ]:
mask, cut_info = cut_mask(org_mask)
new_l, new_t = cut_info
new_fixed_points = np.copy(fixed_points)
new_fixed_points[0,:,0]-=new_l
new_fixed_points[0,:,1]-=new_t
# shadow_mask, rotate_matrix, trans_matrix = mask_to_shadow2(mask, new_fixed_points, rot_rd_low=-30,
#                                                            rot_rd_high=-20,
#                                                            is_vertical_flip=is_vertical_flip,
#                                                            match_legs=is_match_legs)
# Small shadow for flip
shadow_mask, rotate_matrix, trans_matrix = mask_to_shadow2(mask, new_fixed_points, rot_rd_low=-50,
                                                           rot_rd_high=-40,
                                                           is_vertical_flip=is_vertical_flip,
                                                           match_legs=is_match_legs)
plt.imshow(shadow_mask)

In [ ]:
if is_match_legs: #using TPS
    while True:
        shadow_mask_out = match_legs(shadow_mask, fixed_points, rotate_matrix, -0.05, 0.05)
        l,r,t,b = get_mask_bbox(shadow_mask_out)
        if l>0 and t>0 and r<shadow_mask.shape[1]-1 and b<shadow_mask.shape[0]-1:
            shadow_mask = shadow_mask_out
            break
        else:
            print("shadow touch edges")
            break

# Recover original image size
pad_warped_shadow_mask = restore_mask(org_mask, shadow_mask, cut_info)
# plt.imshow(pad_warped_shadow_mask)

In [ ]:
alpha = create_gradient_shadow_alpha_mask(pad_warped_shadow_mask, 
                                          rd_low=0, rd_high=10,
                                          is_vertical_flip=is_vertical_flip,
                                          speed=5, overflow=1.2, offset=120)
revert_shadow_mask=cv2.subtract(img,pad_warped_shadow_mask)

# Convert uint8 to float
revert_shadow_mask_f = revert_shadow_mask.astype(float)
img_f = img.astype(float)

# Multiply the foreground with the alpha matte
foreground_shadow = cv2.multiply(alpha, revert_shadow_mask_f)

# Multiply the background with ( 1 - alpha )
background_img = cv2.multiply(1.0 - alpha, img_f)

# Add the masked foreground and background.
shadow_on_top = cv2.add(foreground_shadow, background_img)
shadow_on_top = shadow_on_top.astype('uint8')

final_background=cv2.subtract(shadow_on_top,org_mask)
final_img = cut_obj+final_background
# plt.imshow(shadow_on_top[...,::-1])
# plt.axis('off')
plt.imshow(final_img[...,::-1])

idx = "flip"
cv2.imwrite(f'{img_name}_{idx}.png', final_img)